## 基本框架
1. 确定回溯参数
2. 确定终止条件
3. 确定每一层循环
4. 剪枝
5. 去重

## 1.组合问题：组合问题一般无序

### LC.77 给定两个整数 n 和 k，返回 1 ... n 中所有可能的 k 个数的组合

In [1]:
def lc77(n,k):
    path,result = [],[]
    def backt(n,k,index):
        ## 终止条件：path中的数等于k个就放在result中
        if len(path) == k:
            result.append(path[:])
            return
        if len(path) > k:
            result
        
        ## 单层循环
        for i in range(index,n+1):
            path.append(i)
            backt(n,k,i+1) # 单个树枝上不允许同一个数出现两次，所以要i+1
            path.pop()
    backt(n,k,1)
    return result
lc77(4,2)

[[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]

### LC.216 找出所有相加之和为 n 的 k 个数的组合。组合中只允许含有 1 - 9 的正整数，并且每种组合中不存在重复的数字

In [2]:
def lc216(n,k):
    path,result = [],[]
    def backt(n,k,index):
        ## 终止条件变成了两个
        if len(path) == k and sum(path) == n:
            result.append(path[:])
            return
        if len(path) > k or sum(path) > n:
            return
        
        for i in range(index,10):
            path.append(i)
            backt(n,k,i+1)
            path.pop()
            
    backt(n,k,1)
    return result
lc216(7,3)

[[1, 2, 4]]

### LC.17 给定一个仅包含数字 2-9 的字符串，返回所有它能表示的字母组合。给出数字到字母的映射如下（与电话按键相同）

In [3]:
def lc17(digits):
    ## 电话上的每个数字对应的字符
    dic = {
            '2':'abc',
            '3':'def',
            '4':'ghi',
            '5':'jkl',
            '6':'mno',
            '7':'pqrs',
            '8':'tuv',
            '9':'wxyz'
        }
    path,result = [],[]
    def backt(n,index):
        if len(path) == n:
            result.append(''.join(path))
            return
        ## 回溯的过程略有不同：往深处回溯是用下一个字符代表的字符组
        for i in dic[digits[index]]:
            path.append(i)
            backt(n,index+1)
            path.pop()
    backt(len(digits),0)
    return result
lc17('23')

['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf']

### LC.39 给定一个无重复元素的数组 candidates 和一个目标数 target ，找出 candidates 中所有可以使数字和为 target 的组合。candidates 中的数字可以无限制重复被选取

In [4]:
def lc39(candidates,target):
    candidates = sorted(candidates)
    path,result = [],[]
    def backt(su,index):
        if su == target:
            result.append(path[:])
            return
        if su > target:
            return
        for i in range(index,len(candidates)):
            path.append(candidates[i])
            su += candidates[i]
            backt(su,i)  ## 允许元素多次使用
            su -= candidates[i]
            path.pop()
    backt(0,0)
    return result
lc39([2,3,5],8),lc39([2,3,6,7],7)

([[2, 2, 2, 2], [2, 3, 3], [3, 5]], [[2, 2, 3], [7]])

### LC.40 给定一个数组 candidates 和一个目标数 target ，找出 candidates 中所有可以使数字和为 target 的组合。candidates 中的每个数字在每个组合中只能使用一次(candidates可能包含重复数字）。

In [5]:
def lc40(candidates,target):
    candidates = sorted(candidates) ## 排序是为了方便去重
    path,result = [],[]
    used = [0 for i in range(len(candidates))] ## 树层去重
    def backt(su,index):
        if su == target:
            result.append(path[:])
            return
        if su > target:
            return
        
        for i in range(index,len(candidates)):
            if i > 0 and candidates[i] == candidates[i-1] and used[i-1] == 0: ## 树层去重
                continue
            path.append(candidates[i])
            su += candidates[i]
            used[i] = 1
            backt(su,i+1) ## 同一个元素不能多次使用
            used[i] = 0
            su -= candidates[i]
            path.pop()
    backt(0,0)
    return result
lc40([10,1,2,7,6,1,5],8)

[[1, 1, 6], [1, 2, 5], [1, 7], [2, 6]]

## 2. 分割问题
### 将符合条件的分割部分加入path

### LC.131 给定一个字符串 s，将 s 分割成一些子串，使每个子串都是回文串。返回 s 所有可能的分割方案。

In [6]:
def lc131(s):
    path,result = [],[]
    def backt(index):
        if index == len(s):
            result.append(path[:])
            return
        if index > len(s):
            return 
        
        for i in range(index+1,len(s)+1): ## 注意下标取值问题
            tmp = s[index:i]
            if tmp == tmp[::-1]: ## 只有回文子串才能加入path
                path.append(tmp)
                backt(i)
                path.pop()
    backt(0)
    return result
lc131("aaba")

[['a', 'a', 'b', 'a'], ['a', 'aba'], ['aa', 'b', 'a']]

### LC.93 给定一个只包含数字的字符串，复原它并返回所有可能的 IP 地址格式。有效的 IP 地址 正好由四个整数（每个整数位于 0 到 255 之间组成，且不能含有前导 0），整数之间用 '.' 分隔。

In [7]:
## 注意排除前导0的情况
def lc93(s):
    path,result = [],[]
    def backt(index):
        if len(path) == 8 and index == len(s): ## 8是加"."的原因
            result.append(''.join(path[:-1]))
            return
        if len(path) > 8 or index > len(s):
            return
        
        for i in range(index+1,len(s)+1):
            tmp = s[index:i]
            if 0<= int(tmp) <= 255 and not(len(tmp)>1 and tmp[0] == '0'): ## 这里排除了前导0
                path.append(tmp)
                path.append('.')
                backt(i)
                path.pop()
                path.pop()
    backt(0)
    return result
lc93("25525511135"),lc93("101023")

(['255.255.11.135', '255.255.111.35'],
 ['1.0.10.23', '1.0.102.3', '10.1.0.23', '10.10.2.3', '101.0.2.3'])

## 3. 子集问题
### 收集每一个节点

### LC.78 给定一组不含重复元素的整数数组 nums，返回该数组所有可能的子集（幂集）。

In [8]:
def lc78(nums):
    path,result = [],[]
    def backt(index):
        result.append(path[:]) ## 不止取叶子节点，每一个节点都取
        if index == len(nums):
            return
        
        for i in range(index,len(nums)):
            path.append(nums[i])
            backt(i+1)
            path.pop()
    backt(0)
    return result
lc78([1,2,3])

[[], [1], [1, 2], [1, 2, 3], [1, 3], [2], [2, 3], [3]]

### LC.90 给定一个可能包含重复元素的整数数组 nums，返回该数组所有可能的子集（幂集）。

In [9]:
def lc90(nums):
    nums = sorted(nums)
    path,result = [],[]
    used = [0 for i in range(len(nums))] ## 树层去重
    def backt(index):
        result.append(path[:])
        if index == len(nums):
            return
        
        for i in range(index,len(nums)):
            if i > 0 and nums[i] == nums[i-1] and used[i-1] == 0:
                continue
            path.append(nums[i])
            used[i] = 1
            backt(i+1)
            used[i] = 0
            path.pop()
    backt(0)
    return result
lc90([1,2,2])

[[], [1], [1, 2], [1, 2, 2], [2], [2, 2]]

### LC.491 给定一个整型数组, 你的任务是找到所有该数组的递增子序列，递增子序列的长度至少是2。

In [10]:
## 不能提前排序，不能提前排序，不能提前排序
def lc491(nums):
    path, result = [],[]
    def backt(index):
        if len(path) >= 2:
            result.append(path[:])
        if index == len(nums):
            return
        
        used = []  ## 记录每一层，到下一层会被重启
        
        for i in range(index,len(nums)):
            if len(path)==0 or (nums[i]>=path[-1] and nums[i] not in used):
                path.append(nums[i])
                used.append(nums[i])
                backt(i+1)
#                 used.pop()  ## 这里不能在pop了，在pop本层就没有了
                path.pop()
            else:
                continue
    backt(0)
    return result
lc491([4, 6, 7, 7])

[[4, 6], [4, 6, 7], [4, 6, 7, 7], [4, 7], [4, 7, 7], [6, 7], [6, 7, 7], [7, 7]]

## 4. 排列问题
### 每次循环都从头开始

### LC.46 给定一个 没有重复 数字的序列，返回其所有可能的全排列。

In [11]:
def lc46(nums):
    path,result = [],[]
    used = [0 for i in range(len(nums))]
    def backt():
        if len(path) == len(nums):
            result.append(path[:])
        if len(path) > len(nums):
            return
        
        for i in range(len(nums)): ## 排列
            if used[i] == 1: ## 记录用过的
                continue
            path.append(nums[i])
            used[i] = 1
            backt()
            used[i] = 0
            path.pop()
    backt()
    return result
lc46([1,2,3])

[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]

### LC.47 给定一个可包含重复数字的序列 nums ，按任意顺序 返回所有不重复的全排列。

In [12]:
def lc47(nums):
    path,result = [],[]
    used1 = [0 for i in range(len(nums))]
    def backt():
        if len(path) == len(nums):
            result.append(path[:])
        if len(path) > len(nums):
            return
        
        used2 = []
        for i in range(len(nums)):
            if used1[i] == 1 or nums[i] in used2: ## used1控制不使用重复数字，used2控制树层去重
                continue
            path.append(nums[i])
            used1[i] = 1
            used2.append(nums[i])
            backt()
            used1[i] = 0
            path.pop()
    backt()
    return result
lc47([1,1,2])

[[1, 1, 2], [1, 2, 1], [2, 1, 1]]

## 5. 应用题

### LC.332 给你一份航线列表 tickets ，其中 tickets[i] = [fromi, toi] 表示飞机出发和降落的机场地点。请你对该行程进行重新规划排序。所有这些机票都属于一个从 JFK（肯尼迪国际机场）出发的先生，所以该行程必须从 JFK 开始。如果存在多种有效的行程，请你按字典排序返回最小的行程组合。

In [13]:
from collections import defaultdict
def lc332(tickets):
    dic = defaultdict(list) ## 返回值为空时返回一个[],dict中的每一个单元都是list,所以后面用list
    
    ## 将机票情况加入并排序
    for start,end in tickets:
        dic[start].append(end)
    for key in dic:
        dic[key] = sorted(dic[key])
#     print(dic)
    
    result = ['JFK']
    path = []
    s = 'JFK'
    def backt(s):
#         print(result)
        if s == [] and len(result) < len(tickets) + 1:
            return
        if len(result) == len(tickets)+1:
            path.append(result[:])
            return True
        print(result)
        for i in range(len(dic[s])):
            end = dic[s].pop(i)
            result.append(end)
            if backt(end):return True   ## 防止有多个解，产生超时   
            result.pop()
            dic[s].append(end)
            dic[s] = sorted(dic[s])
            
    backt(s)
    return path
    
tickets = [["JFK","SFO"],["JFK","ATL"],["SFO","ATL"],["ATL","JFK"],["ATL","SFO"]]
t2 = [["JFK","KUL"],["JFK","NRT"],["NRT","JFK"]]
t3 = [["AXA","EZE"],["EZE","AUA"],["ADL","JFK"],["ADL","TIA"],["AUA","AXA"],["EZE","TIA"],["EZE","TIA"],["AXA","EZE"],["EZE","ADL"],["ANU","EZE"],["TIA","EZE"],["JFK","ADL"],["AUA","JFK"],["JFK","EZE"],["EZE","ANU"],["ADL","AUA"],["ANU","AXA"],["AXA","ADL"],["AUA","JFK"],["EZE","ADL"],["ANU","TIA"],["AUA","JFK"],["TIA","JFK"],["EZE","AUA"],["AXA","EZE"],["AUA","ANU"],["ADL","AXA"],["EZE","ADL"],["AUA","ANU"],["AXA","EZE"],["TIA","AUA"],["AXA","EZE"],["AUA","SYD"],["ADL","JFK"],["EZE","AUA"],["ADL","ANU"],["AUA","TIA"],["ADL","EZE"],["TIA","JFK"],["AXA","ANU"],["JFK","AXA"],["JFK","ADL"],["ADL","EZE"],["AXA","TIA"],["JFK","AUA"],["ADL","EZE"],["JFK","ADL"],["ADL","AXA"],["TIA","AUA"],["AXA","JFK"],["ADL","AUA"],["TIA","JFK"],["JFK","ADL"],["JFK","ADL"],["ANU","AXA"],["TIA","AXA"],["EZE","JFK"],["EZE","AXA"],["ADL","TIA"],["JFK","AUA"],["TIA","EZE"],["EZE","ADL"],["JFK","ANU"],["TIA","AUA"],["EZE","ADL"],["ADL","JFK"],["ANU","AXA"],["AUA","AXA"],["ANU","EZE"],["ADL","AXA"],["ANU","AXA"],["TIA","ADL"],["JFK","ADL"],["JFK","TIA"],["AUA","ADL"],["AUA","TIA"],["TIA","JFK"],["EZE","JFK"],["AUA","ADL"],["ADL","AUA"],["EZE","ANU"],["ADL","ANU"],["AUA","AXA"],["AXA","TIA"],["AXA","TIA"],["ADL","AXA"],["EZE","AXA"],["AXA","JFK"],["JFK","AUA"],["ANU","ADL"],["AXA","TIA"],["ANU","AUA"],["JFK","EZE"],["AXA","ADL"],["TIA","EZE"],["JFK","AXA"],["AXA","ADL"],["EZE","AUA"],["AXA","ANU"],["ADL","EZE"],["AUA","EZE"]]
lc332(t2)

['JFK']
['JFK', 'KUL']
['JFK', 'NRT']
['JFK', 'NRT', 'JFK']


[['JFK', 'NRT', 'JFK', 'KUL']]

### LC.51 按照国际象棋的规则，皇后可以攻击与之处在同一行或同一列或同一斜线上的棋子。n 皇后问题 研究的是如何将 n 个皇后放置在 n×n 的棋盘上，并且使皇后彼此之间不能相互攻击。给你一个整数 n ，返回所有不同的 n 皇后问题 的解决方案。每一种解法包含一个不同的 n 皇后问题 的棋子放置方案，该方案中 'Q' 和 '.' 分别代表了皇后和空位。

In [14]:
def lc51(n):
    def valid(row,col,result,n):
        ## 列
        num1 = 0
        for i in range(n):
            if result[row][i] == 'Q':
                num1 += 1
        if num1 > 0:
            return False
        
        ## 行
        num2 = 0
        for i in range(n):
            if result[i][col] == 'Q':
                num2 += 1
        if num2 > 0:
            return False
        
        ## 正对角
        num3 = 0
        for i in range(-n,n):
            if 0<=row+i<n and 0<=col-i<n and result[row+i][col-i] == 'Q':
                num3 += 1
        if num3 > 0:
            return False
        
        ## 逆对角
        num4 = 0
        for i in range(-n,n):
            if 0<=row+i<n and 0<=col+i<n and result[row+i][col+i] == 'Q':
                num4 += 1
        if num4 > 0:
            return False
#         print(row,col,result,num1,num2,num3,num4)
        return True
        
    results = []
    result = [['.' for i in range(n)] for j in range(n)]
    def backt(n,row,result):
#         print(result,row)
        if row == n:
            results.append([''.join(result[i]) for i in range(len(result))])
            return
        for i in range(n):
            if valid(row,i,result,n):
                result[row][i] = 'Q'
                backt(n,row+1,result)
                result[row][i] = '.'
    backt(n,0,result)
    return results
lc51(4),lc51(5)

([['.Q..', '...Q', 'Q...', '..Q.'], ['..Q.', 'Q...', '...Q', '.Q..']],
 [['Q....', '..Q..', '....Q', '.Q...', '...Q.'],
  ['Q....', '...Q.', '.Q...', '....Q', '..Q..'],
  ['.Q...', '...Q.', 'Q....', '..Q..', '....Q'],
  ['.Q...', '....Q', '..Q..', 'Q....', '...Q.'],
  ['..Q..', 'Q....', '...Q.', '.Q...', '....Q'],
  ['..Q..', '....Q', '.Q...', '...Q.', 'Q....'],
  ['...Q.', 'Q....', '..Q..', '....Q', '.Q...'],
  ['...Q.', '.Q...', '....Q', '..Q..', 'Q....'],
  ['....Q', '.Q...', '...Q.', 'Q....', '..Q..'],
  ['....Q', '..Q..', 'Q....', '...Q.', '.Q...']])

### LC.37 编写一个程序，通过填充空格来解决数独问题。一个数独的解法需遵循如下规则： 数字 1-9 在每一行只能出现一次。 数字 1-9 在每一列只能出现一次。 数字 1-9 在每一个以粗实线分隔的 3x3 宫内只能出现一次。 空白格用 '.' 表示。

In [15]:
import time

def lc37(board):
    num_list = [str(i) for i in range(1,10)]
    
    def valid(row,col,board,number):
        if board[row][col] != '.': return False
        ## 行
        for i in range(9):
            if board[row][i] == number:return False
        ## 列
        for i in range(9):
            if board[i][col] == number:return False
        ## 九宫格
        r,c = (row//3)*3,(col//3)*3
        for i in range(r,r+3):
            for j in range(c,c+3):
                if board[i][j] == number:
                    return False
        return True
    
    def backt(board):
#         print(board)
        for i in range(9):
            for j in range(9):
                if board[i][j] != '.':continue ## 这里需要注意
                for number in num_list:
                    # print(i,j,number)
                    if valid(i,j,board,number):
                        board[i][j] = number
                        if backt(board):return True
                        board[i][j] = '.'
                return False
        return True
    backt(board)
    return board

board = [["5","3",".",".","7",".",".",".","."],["6",".",".","1","9","5",".",".","."],[".","9","8",".",".",".",".","6","."],["8",".",".",".","6",".",".",".","3"],["4",".",".","8",".","3",".",".","1"],["7",".",".",".","2",".",".",".","6"],[".","6",".",".",".",".","2","8","."],[".",".",".","4","1","9",".",".","5"],[".",".",".",".","8",".",".","7","9"]]
lc37(board)

[['5', '3', '4', '6', '7', '8', '9', '1', '2'],
 ['6', '7', '2', '1', '9', '5', '3', '4', '8'],
 ['1', '9', '8', '3', '4', '2', '5', '6', '7'],
 ['8', '5', '9', '7', '6', '1', '4', '2', '3'],
 ['4', '2', '6', '8', '5', '3', '7', '9', '1'],
 ['7', '1', '3', '9', '2', '4', '8', '5', '6'],
 ['9', '6', '1', '5', '3', '7', '2', '8', '4'],
 ['2', '8', '7', '4', '1', '9', '6', '3', '5'],
 ['3', '4', '5', '2', '8', '6', '1', '7', '9']]